In [1]:
import matplotlib.pyplot as plt 
import seaborn as sns
import numpy as np 
import os 
import cv2
import pickle
import pandas as pd
import itertools
import tensorflow as tf
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix

%matplotlib inline

In [2]:
cm_plot_labels = ['Heart','Oblong','Oval','Round', 'Square']
fig_path = 'C:\\Users\\tprak_o7ept1f\\OneDrive\\Desktop\\Face_Shape_Classification\\images\\'

def create_confusion_matrix(y_test_labels, y_preds, classes, title='Confusion Matrix', 
                            normalize=False, cmap=plt.cm.Blues):

    cm = confusion_matrix(y_test_labels, y_preds)
    
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    # print(cm)

    plt.figure(figsize=(16,8))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title, fontsize = 15)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.savefig(f"{fig_path+title}.png");    # for saving images to .png file
    plt.show()

In [3]:
y_label_dict = {0: 'Heart', 1: 'Oblong', 2: 'Oval', 3: 'Round', 4: 'Square'}

def show_X_img(num, index_array, X_array, y_actual, fig_title, ncols=1):
    '''this function display images from an image array'''
    ncols= ncols
    nrows= int(num/ncols)
    fig, ax = plt.subplots(nrows, ncols, figsize =(ncols*4,nrows*4))
    fig.suptitle(fig_title, size = 20)
    ax = ax.ravel()
    for i, index in enumerate(index_array):
        img = X_array[index] * 255
        img = np.asarray(img, int)
        label = y_label_dict[y_actual[index]]
        ax[i].imshow(img, cmap='gray')
        ax[i].set_title(label, size=16)

In [5]:
def compare_misclass(df, predict, fig_title='Comparing Misclassification', ax_title=''):
    fig, ax = plt.subplots(nrows=1, ncols=2, sharey=True, figsize=(15,5))
    fig.suptitle(fig_title, fontsize=16)
    ax[0].hist(df.y_actual, bins=9, color='lightcoral')
    ax[0].set_xticks(range(0,5))
    ax[0].set_xticklabels(cm_plot_labels)
    ax[0].set_title(f'{ax_title}\n ACTUAL CLASS')
    ax[1].hist(df[predict], bins=9, color='mediumturquoise')
    ax[1].set_xticks(range(0,5))
    ax[1].set_xticklabels(cm_plot_labels)
    ax[1].set_title(f'{ax_title}\n PREDICTED CLASS')
    plt.savefig(f"{fig_path+fig_title}.png")

In [7]:
def plot_misclass_img_proba(df, list_index):
  for i in list_index:
    img = X_test[i] * 255
    img = np.asarray(img, int)
    label = y_label_dict[y_actual[i]]

    plt.figure(figsize=(15,5))
    plt.subplot(1,3,1)
    plt.imshow(img, cmap='gray')
    plt.title(label, size=16)
    plt.subplot(1,3,2)
    df.loc[i, ['heart_s', 'oblong_s', 'oval_s', 'round_s','square_s']].plot(kind='bar', color='pink')
    plt.title('Probabilities - CNN from scratch', fontsize=14, y=1.01)
    plt.subplot(1,3,3)
    df.loc[i, ['heart_t', 'oblong_t', 'oval_t', 'round_t','square_t']].plot(kind='bar', color='plum')
    plt.title('Probabilities - CNN transfer learning', fontsize=14, y=1.01)

In [8]:
path = "C:\\Users\\tprak_o7ept1f\\OneDrive\\Desktop\\Face_Shape_Classification\\"

X_train = np.asarray(pickle.load(open(path + "dataX_train_rgb.pickle","rb")))
y_train = np.asarray(pickle.load(open(path + "datay_train_rgb.pickle","rb")))
X_test = np.asarray(pickle.load(open(path + "dataX_test_rgb.pickle","rb")))
y_test = np.asarray(pickle.load(open(path + "datay_test_rgb.pickle","rb")))

In [9]:
print("Data Summary")
print("--------------------")
print(f"X_train shape {X_train.shape}")
print(f"y_train shape {y_train.shape}")
print("--------------------")
print(f"X_test shape {X_test.shape}")
print(f"y_test shape {y_test.shape}")

Data Summary
--------------------
X_train shape (4000, 224, 224, 3)
y_train shape (4000, 5)
--------------------
X_test shape (1000, 224, 224, 3)
y_test shape (1000, 5)


In [11]:
transfer_path = "C:\\Users\\tprak_o7ept1f\\OneDrive\\Desktop\\Face_Shape_Classification\\saved_models\\"
transfer_file = transfer_path + 'vgg16-face-2.h5'
transfer_file1 = transfer_path + 'vgg16-face-1.h5'
mod_transfer = tf.keras.models.load_model(transfer_file)
mod_transfer1 = tf.keras.models.load_model(transfer_file1)

In [12]:
# Make predictions
y_actual = np.argmax(y_test, axis=-1)
y_predict_scratch = np.argmax(mod_transfer1.predict(X_test), axis=1)
y_predict_transfer = np.argmax(mod_transfer.predict(X_test), axis=1)

32/32 ━━━━━━━━━━━━━━━━━━━━ 96s 3s/step
32/32 ━━━━━━━━━━━━━━━━━━━━ 90s 3s/step


In [13]:
# Generate predict probabilities
predict_proba_scratch = mod_transfer1.predict(X_test)
predict_proba_transfer = mod_transfer.predict(X_test)

32/32 ━━━━━━━━━━━━━━━━━━━━ 102s 3s/step
32/32 ━━━━━━━━━━━━━━━━━━━━ 111s 3s/step


In [14]:
mod_transfer1.evaluate(X_test, y_test)

32/32 ━━━━━━━━━━━━━━━━━━━━ 107s 3s/step - accuracy: 0.9126 - loss: 0.4304


[0.3667786121368408, 0.9120000004768372]

In [15]:
mod_transfer.evaluate(X_test, y_test)

32/32 ━━━━━━━━━━━━━━━━━━━━ 109s 3s/step - accuracy: 0.8964 - loss: 0.5279


[0.4422782063484192, 0.9079999923706055]

In [16]:
def create_df(y_actual, y_predict, predict_proba, X_test, y_test):
  '''this function creates a dataframe with predictions and probabilities of a model'''
  # generate predictions

  actual = pd.DataFrame(y_actual, columns=['y_actual'])
  predict = pd.DataFrame(y_predict, columns=['y_predict'])

  # generate prediction probabilities 

  probability_list = []
  for i, item in enumerate(predict_proba):
    probabilities = {}
    probabilities['heart'] = round(item[0] * 100,2)
    probabilities['oblong'] = round(item[1] * 100,2)
    probabilities['oval'] = round(item[2] *100,2)
    probabilities['round'] = round(item[3] *100,2)
    probabilities['square'] = round(item[4] *100,2)
    probability_list.append(probabilities)
  proba = pd.DataFrame(probability_list)

  # create dataframe
  df = pd.concat([actual, predict, proba],axis=1)
  
  return df

In [17]:
transfer1 = create_df(y_actual, y_predict_scratch, predict_proba_scratch, X_test, y_test)

transfer = create_df(y_actual, y_predict_transfer, predict_proba_transfer, X_test, y_test)

In [19]:
# dataframe for model with transfer learning
transfer.columns = [x + '_t' for x in transfer.columns]
transfer.head()

,y_actual_t,y_predict_t,heart_t,oblong_t,oval_t,round_t,square_t
0,2,3,0.00,0.0,1.77,98.120003,0.12
1,0,2,9.50,0.0,90.50,0.000000,0.00
2,3,3,0.00,0.0,0.00,100.000000,0.00
3,3,3,0.02,0.0,0.00,99.540001,0.44
4,0,0,100.00,0.0,0.00,0.000000,0.00


In [20]:
# combine the data frame for evaluations
evaluations = pd.concat([scratch, transfer], axis=1)

# drop one of the y_actual as they are the same
evaluations = evaluations.drop(columns='y_actual_t')
evaluations = evaluations.rename(columns={'y_actual_s': 'y_actual'})

# create columns with actual labels
evaluations['actual'] = evaluations['y_actual'].map(y_label_dict)
evaluations['s_predict'] = evaluations['y_predict_s'].map(y_label_dict)
evaluations['t_predict'] = evaluations['y_predict_t'].map(y_label_dict)

# create new columns to detect where the 2 models misclassify, and the differences in predictions
evaluations['predict_diff'] = evaluations['y_predict_s'] - evaluations['y_predict_t']
evaluations['t_misclass'] = evaluations['y_actual'] - evaluations['y_predict_t']
evaluations['s_misclass'] = evaluations['y_actual'] - evaluations['y_predict_s']

evaluations

NameError: name 'scratch' is not defined